In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH 
asrt_list AS (
    SELECT DISTINCT 
        a.store_id, 
        CASE
            WHEN c.business_unit = '1500' THEN 'Supermarket'
            WHEN c.business_unit = '2000' AND c.concept IN ('WiN','Rural') THEN c.concept
            ELSE 'Urban'
        END business_model,
        a.product_id,
        b.manufacturer_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
    WHERE c.business_unit IN ('1500', '2000', '6000')
        AND c.store_status_desc = "Đang hoạt động"
        AND UPPER(c.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(c.store_name) NOT LIKE 'OUTLET%' 
        AND UPPER(c.store_name) NOT LIKE '%HUB%' 
        AND UPPER(c.store_name) NOT LIKE '%ONLINE%'
        AND UPPER(c.store_name) NOT LIKE '%KHO %'
        AND UPPER(c.store_name) NOT LIKE '%B2B%'
        AND a.store_id NOT LIKE 'GH%'
        AND b.mch2_id IN ('102','201','202','203')
),
asm_kg AS (
    SELECT DISTINCT st.business_unit, a.store_id, a.product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st USING(store_id)
    WHERE (
            (st.business_unit = '2000' AND a.assortment IN ('VMP_KGMB', 'VMP_KGMN', 'WMP_THOIVU'))
            OR (st.business_unit = '1500' AND (a.assortment LIKE 'KG\\_%' OR a.assortment LIKE 'VM\\_KG%'))
            OR a.assortment LIKE 'KG_%'
        )
        AND a.status = 'ACTIVE'
        AND DATE(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
        AND DATE(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
), 
main AS (
    SELECT DISTINCT
        b.business_unit,
        a.store_id,
        b.store_name,
        a.business_model,
        b.region_domain_vn mien,
        b.city,
        c.mch2_id,
        a.product_id,
        c.product_name,
        a.manufacturer_id,
        c.manufacturer_name,
        d.core_range assortment_type
    FROM asrt_list a 
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment d
        ON a.store_id = d.store_id AND a.product_id = d.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc e 
        ON a.store_id = e.store_id AND a.product_id = e.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_fresh e1
    ON a.store_id = e1.store_id AND a.product_id = e1.product_id
    LEFT JOIN asm_kg f
        ON a.store_id = f.store_id AND a.product_id = f.product_id
    WHERE e.store_id IS NULL AND f.store_id IS NULL AND
     e1.store_id IS NULL
)
SELECT *
FROM main;
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd_his(
    calday DATE,
    business_unit STRING,
    store_id STRING, 
    business_model STRING,
    mch2_id STRING,
    product_id STRING,
    assortment_type STRING,
    manufacturer_id STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd_his 
WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    business_unit,
    store_id,
    business_model,
    mch2_id,
    product_id,
    assortment_type,
    manufacturer_id
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd;
""")